## Keyword Search (Sparse Vector Search)

インストール
```
pip install "transformers==4.45" "flagembedding==1.3.5" "tokenizers==0.20.3"
pip install qdrant-client # Sparse Vector DB
pip install torch
```

### 利用ライブラリの解説

| ライブラリ | 役割 | インストール |
|---|---|---|
| **FlagEmbedding** | BGE-M3のSparse/Denseエンコード（BAAI公式） | pip install FlagEmbedding |
| **qdrant-client** | ベクトルDBクライアント。Sparseベクトルの永続化・検索 | pip install qdrant-client |
| **torch** | FlagEmbeddingの依存ライブラリ | pip install torch |

Qdrantはローカルフォルダに永続化できるため、サーバー不要でファイルベースのDBとして利用できます。

### 開発元/ライセンス

| | BGE-M3 | Qdrant |
|---|---|---|
| 開発元 | BAAI（中国・研究機関） | Qdrant GmbH（ドイツ・企業） |
| ライセンス | MIT | Apache 2.0 |
| 商用利用 | ✅ | ✅ |
| 無償利用 | ✅ | ✅（セルフホスト） |

<div style="text-align: center">BGE-M3 https://huggingface.co/BAAI/bge-m3</div>  

### 処理フロー概要

#### インデックス作成
```
コーパス
 └─ BGE-M3エンコード → Sparseベクトル（{token_id: weight}）
     |  Qdrant 初期化 <- ローカルに作成すれば、次回起動時もデータが残る
     |  Qdrant コレクション作成
     └─ Qdrant（ローカルディスク）にアップサート
```
ローカルに作成すれば、2回目以降の起動では **Step 2でDBを読み込み、Step 3はスキップ** されるため、エンコードとアップサートをやり直す必要がなくなります。

#### クエリ
```
クエリ
 └─ BGE-M3エンコード → Sparseベクトル
     └─ Qdrant検索 → スコア順で結果返却
```

In [1]:
import sys
from importlib.metadata import version
import numpy as np

from FlagEmbedding import BGEM3FlagModel
from qdrant_client import QdrantClient
from qdrant_client.models import (
    SparseVectorParams,
    SparseIndexParams,
    PointStruct,
    SparseVector,
    NamedSparseVector,
)
print(sys.version)
print(f"transformers={version('transformers')}, flagembedding={version('flagembedding')}, " +
      f"tokenizers={version('tokenizers')}, qdrant_client={version('qdrant-client')}, ")

# ============================================================
# 設定
# ============================================================
COLLECTION_NAME = "notes"              # Qdrantコレクション名
PERSIST_DIR     = "./qdrant_storage"   # 永続化ディレクトリ（ローカルフォルダ）
SPARSE_NAME     = "bgem3_sparse"       # Sparseベクトルのフィールド名

# ============================================================
# コーパス（インデックス対象のテキスト）
# ============================================================
corpus = [
    '順に秤量し、ビーカーに加える。各秤量値は実験ノートに記録しておく。',
    'スターラーで軽く攪拌して溶解・混合を促進する。',
    '順番に加え、それぞれ秤量値を記録する。',
    '同じビーカー内で再度スターラー攪拌を行う。',
    'ビーカーを超音波分散機にセットし分散処理を行う。'
]

# ============================================================
# Step 0: BGE-M3モデルのロード
#   - use_fp16=True: 半精度演算でメモリ・速度を最適化
#   - CPUのみの環境では use_fp16=False に変更
# ============================================================
print("Step 0: BGE-M3モデルをロード中...")
model = BGEM3FlagModel("BAAI/bge-m3", use_fp16=True)

3.13.12 (tags/v3.13.12:1cbe481, Feb  3 2026, 18:22:25) [MSC v.1944 64 bit (AMD64)]
transformers=4.45.0, flagembedding=1.3.5, tokenizers=0.20.3, qdrant_client=1.16.2, 
Step 0: BGE-M3モデルをロード中...


Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

## Create index

In [2]:
# ============================================================
# Create index
print("\n### Create index ###\n")
# ============================================================
# Step 1: BGE-M3でSparseエンコード
#   - return_dense=False     : Denseベクトルは今回不要
#   - return_sparse=True     : Sparseベクトルを取得
#   - return_colbert_vecs=False : ColBERTも今回不要
#   出力: lexical_weights = [{token_id(str): weight(float), ...}, ...]
# ============================================================
print("Step 1: コーパスをSparseエンコード中...")
encoded = model.encode(
    corpus,               # リスト形式
    return_dense=False,   # Denseベクトルは不要
    return_sparse=True,   # Sparseベクトルのみ取得
    return_colbert_vecs=False, # ColBERTベクトルは不使用
    batch_size=4,
)
lexical_weights = encoded["lexical_weights"]  # 各文書のSparseベクトル（辞書形式）

# エンコード結果の確認（どのトークンに重みが付いたか）
print("\n--- エンコード結果サンプル（doc[0]の上位5トークン）---")
sample = sorted(lexical_weights[0].items(), key=lambda x: -x[1])[:5]
for token_id, weight in sample:
    # token_idからトークン文字列に変換して表示
    token_str = model.tokenizer.convert_ids_to_tokens([int(token_id)])
    print(f"  token: {token_str[0]:15s}  weight: {weight:.4f}")

# ============================================================
# Step 2: Qdrantクライアントの初期化（永続化モード/インメモリ）
#   - path=PERSIST_DIR を指定するとローカルフォルダにDBを保存
#   - 次回起動時も同じpathを指定すればデータが残っている
#   （インメモリにする場合は QdrantClient(":memory:") に変更）
# ============================================================
#print(f"\nStep 2: Qdrantクライアントを初期化（永続化先: {PERSIST_DIR}）")
#client = QdrantClient(path=PERSIST_DIR)
print(f"\nStep 2: Qdrantクライアントを初期化（インメモリで使用）")
client = QdrantClient(":memory:")

# ============================================================
# Step 3: コレクションの作成（初回のみ）
#   - 既に存在する場合はスキップ
#   - SparseVectorParams: Sparseベクトル専用のフィールド設定
#   - vectors_config={}: Denseベクトルは使用しないため空
# ============================================================
existing = [c.name for c in client.get_collections().collections]

if COLLECTION_NAME not in existing:
    print(f"Step 3: コレクション '{COLLECTION_NAME}' を新規作成")
    client.create_collection(
        collection_name=COLLECTION_NAME,
        vectors_config={},          # Denseなし
        sparse_vectors_config={
            SPARSE_NAME: SparseVectorParams(
                # インデックスもディスクに永続化、インメモリの場合は無視される
                index=SparseIndexParams(on_disk=True)
            )
        }
    )
else:
    print(f"Step 3: コレクション '{COLLECTION_NAME}' は既に存在 → スキップ")

# ============================================================
# Step 4: ドキュメントをQdrantにアップサート（追加 or 更新）
#   - PointStruct: Qdrantの1レコード単位
#     - id      : ドキュメントID（整数）
#     - vector  : Sparseベクトル（indices + values）
#     - payload : 元テキスト等のメタデータ（検索結果で取得可能）
#   - SparseVector: {indices: [token_id, ...], values: [weight, ...]}
# ============================================================
print("Step 4: ドキュメントをQdrantにアップサート中...")
points = []
for idx, (text, lw) in enumerate(zip(corpus, lexical_weights)):
    indices = [int(k)   for k in lw.keys()]
    values  = [float(v) for v in lw.values()]
    points.append(
        PointStruct(
            id=idx,
            vector={
                SPARSE_NAME: SparseVector(indices=indices, values=values)
            },
            payload={"text": text, "doc_id": idx}
        )
    )

client.upsert(collection_name=COLLECTION_NAME, points=points)
print(f"  → {len(points)} 件のドキュメントを保存しました")


### Create index ###

Step 1: コーパスをSparseエンコード中...


pre tokenize: 100%|█████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 697.37it/s]
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Inference Embeddings: 100%|██████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  2.71it/s]


--- エンコード結果サンプル（doc[0]の上位5トークン）---
  token: 秤                weight: 0.2553
  token: 実験               weight: 0.2403
  token: カー               weight: 0.2383
  token: ノート              weight: 0.2167
  token: 量                weight: 0.2130

Step 2: Qdrantクライアントを初期化（インメモリで使用）
Step 3: コレクション 'notes' を新規作成
Step 4: ドキュメントをQdrantにアップサート中...
  → 5 件のドキュメントを保存しました


## Query

In [3]:
# ============================================================
# Query
print("\n### Query ###\n")
# ============================================================
# Query Step 1: クエリのエンコード
#   - コーパスと同じモデル・設定でエンコードする
# ============================================================
query = '攪拌混合'
print(f"\nQuery Step 1: クエリをエンコード中... Query: '{query}'")

# BGE-M3でエンコード
q_encoded = model.encode(
    [query],              # リスト形式で渡す（バッチ処理のため）
    return_dense=False,   # Denseベクトルは不要
    return_sparse=True,   # Sparseベクトルのみ取得
    return_colbert_vecs=False,
)

# lexical_weights（{token_id: weight}の辞書）を取得
q_lw      = q_encoded["lexical_weights"][0]
q_indices = [int(k)   for k in q_lw.keys()]
q_values  = [float(v) for v in q_lw.values()]

print("  クエリの展開トークン（上位5件）:")
top_tokens = sorted(q_lw.items(), key=lambda x: -x[1])[:5]
for token_id, weight in top_tokens:
    token_str = model.tokenizer.convert_ids_to_tokens([int(token_id)])
    print(f"    {token_str[0]:15s}  weight: {weight:.4f}")

# ============================================================
# Query Step 2: Sparseベクトル検索
#   - NamedSparseVector: フィールド名 + SparseVectorを指定
#   - limit: 上位何件を返すか
#   - with_payload=True: テキスト等のメタデータも返す
# ============================================================
print("\nQuery Step 2: 検索実行中...")
results = client.query_points(
    collection_name=COLLECTION_NAME,
    query=SparseVector(indices=q_indices, values=q_values),
    using=SPARSE_NAME,
    limit=3,
    with_payload=True,
)

# ============================================================
# Query Step 3: 結果表示
# ============================================================
print(f"\n=== 検索結果（Query: '{query}'）===")
for rank, hit in enumerate(results.points, start=1):
    # tupleの場合 hit[0] が ScoredPoint
    point = hit[0] if isinstance(hit, tuple) else hit
    print(f"  [{rank}位] score={point.score:.4f} | {point.payload['text']}")



### 実行結果イメージ
dmy = '''
Step 1: BGE-M3モデルをロード中...
Step 2: コーパスをSparseエンコード中...

--- エンコード結果サンプル（doc[0]の上位5トークン）---
  token: ビーカー         weight: 1.2341
  token: 秤量            weight: 1.1823
  token: 記録            weight: 0.9412
  ...

Step 3: Qdrantクライアントを初期化（永続化先: ./qdrant_storage）
Step 4: コレクション 'lab_notes' を新規作成
Step 5: ドキュメントをQdrantにアップサート中...
  → 5 件のドキュメントを保存しました

Step 6: クエリをエンコード中... Query: '攪拌混合'
  クエリの展開トークン（上位5件）:
    攪拌            weight: 1.8923
    混合            weight: 1.7341
    スターラー       weight: 0.8231  ← 語彙拡張
    ...

=== 検索結果（Query: '攪拌混合'）===
  [1位] score=2.4821 | スターラーで軽く攪拌して溶解・混合を促進する。
  [2位] score=1.8234 | 同じビーカー内で再度スターラー攪拌を行う。
  [3位] score=0.3241 | 順に秤量し、ビーカーに加える。...
'''


### Query ###


Query Step 1: クエリをエンコード中... Query: '攪拌混合'
  クエリの展開トークン（上位5件）:
    攪拌               weight: 0.3216
    混合               weight: 0.2939
    ▁                weight: 0.0586

Query Step 2: 検索実行中...

=== 検索結果（Query: '攪拌混合'）===
  [1位] score=0.1586 | スターラーで軽く攪拌して溶解・混合を促進する。
  [2位] score=0.0922 | 同じビーカー内で再度スターラー攪拌を行う。
  [3位] score=0.0013 | 順番に加え、それぞれ秤量値を記録する。
